In [ ]:
pip install openai

In [ ]:
import os
import openai
#from google.colab import userdata

os.environ["OPENAI_API_KEY"] = "sk-proj-C09Y"

In [ ]:
def generate_post(topic):
    prompt_post = f"Напишите подробный пост для блога на тему: {topic}."
    response_post = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_post}],
        max_tokens=50,
        n=1,
        stop=None,
        temperature=0.7,
    )
    post_content = response_post.choices[0].message.content.strip()

    return post_content

In [ ]:
topics = [
    "Преимущества медитации",
    "Здоровое питание для занятых людей",
    "Советы по управлению временем",
    "Как начать свой бизнес",
    "Путешествия по бюджету"
]

In [ ]:
for topic in topics:
    generated_post = generate_post(topic)
    print(f"Тема: {topic}\n")
    print(generated_post)
    print("---")

In [ ]:
def generate_post(topic):
    prompt_title = f"Придумайте привлекательный заголовок для поста на тему: {topic}"
    response_title = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_title}],
        max_tokens=50,
        n=1,
        temperature=0.7,
    )
    title = response_title.choices[0].message.content.strip()

    prompt_meta = f"Напишите краткое, но информативное мета-описание для поста с заголовком: {title}"
    response_meta = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_meta}],
        max_tokens=100,
        n=1,
        temperature=0.7,
    )
    meta_description = response_meta.choices[0].message.content.strip()

    prompt_post = f"Напишите подробный и увлекательный пост для блога на тему: {topic}. Используйте короткие абзацы, подзаголовки, примеры и ключевые слова для лучшего восприятия и SEO-оптимизации."
    response_post = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt_post}],
        max_tokens=2048,
        n=1,
        temperature=0.7,
    )
    post_content = response_post.choices[0].message.content.strip()

    return {
        "title": title,
        "meta_description": meta_description,
        "post_content": post_content
    }

In [ ]:
post = generate_post(topic)
print(post)

In [ ]:
import pprint
pprint.pprint(post, compact=True)

In [ ]:
import requests
api_key="sk-9d4"
response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/generate/sd3",
    headers={
        "authorization": f"Bearer {api_key}", #sk-9d4a
        "accept": "image/*"
    },
    files={"none": ''},
    data={
        "prompt": "cat ninja with sword, redy to firght", # промпт
        "output_format": "jpeg", # Меняем формат "webp" на "jpeg"
    },
)

if response.status_code == 200:
    with open("./story.jpg", 'wb') as file:
        file.write(response.content)
else:
    raise Exception(str(response.json()))

In [ ]:
!pip install pillow

In [ ]:
from PIL import Image, ImageDraw, ImageFont
text = generate_post("Ninja Cat, Story of Greate fighter")


In [45]:
img = Image.open('story.jpg').convert('RGBA')
draw = ImageDraw.Draw(img)
font = ImageFont.truetype('FreeMono.ttf', 65)


In [47]:

img_width, img_height = img.size
x, y = 100, 500
# Разбиваем текст на строки по переносу
lines = []
line = ""
for word in text.get('title').split():
    if draw.textlength(line + word, font) <= img_width - 200:
        line += word + " "
    else:
        lines.append(line.strip())
        line = word + " "
lines.append(line.strip())

# Получаем высоту строки
ascent, descent = font.getmetrics()
line_height = ascent + descent

# Получаем размеры текста
text_width = max(draw.textlength(line, font) for line in lines)
text_height = line_height * len(lines)

In [49]:
overlay = Image.new('RGBA', (int(text_width), int(text_height)), (0, 0, 128, 180))
img.paste(overlay, (int(x), int(y)), mask=overlay)
for line in lines:
    draw.text((x, y), line, font=font, fill=(255, 0, 0))
    y += line_height
img = img.convert("RGB")
img.save("story_text.jpg")

In [51]:
import requests
from PIL import Image
# Запрос к API для генерации изображения
response = requests.post(
    f"https://api.stability.ai/v2beta/stable-image/generate/sd3",
    headers={
        "authorization": f"Bearer {api_key}",
        "accept": "image/*"
    },
    files={"none": ''},
    data={
        "prompt": "Cat samurai with sword in mask, ready to fight",
        "output_format": "jpeg",
    },
)
# Проверка успешного ответа
if response.status_code == 200:
    # Сохранение сгенерированного изображения
    with open("./story.jpg", 'wb') as file:
        file.write(response.content)
    # Открытие изображения с помощью Pillow
    with Image.open("./story.jpg") as img:
        # Получение текущих размеров изображения
        width, height = img.size
        # Целевое соотношение 9:16
        target_aspect_ratio = 9 / 16
        current_aspect_ratio = width / height
        if current_aspect_ratio > target_aspect_ratio:
            # Ширина изображения слишком большая, нужно обрезать по ширине
            new_width = int(height * target_aspect_ratio)
            left = (width - new_width) // 2
            right = left + new_width
            cropped_img = img.crop((left, 0, right, height))
        elif current_aspect_ratio < target_aspect_ratio:
            # Высота изображения слишком большая, нужно обрезать по высоте
            new_height = int(width / target_aspect_ratio)
            top = (height - new_height) // 2
            bottom = top + new_height
            cropped_img = img.crop((0, top, width, bottom))
        else:
            # Изображение уже в нужных пропорциях
            cropped_img = img
        # Сохранение обрезанного изображения
        cropped_img.save("./story_cropped.jpg")
        print("Изображение успешно обрезано и сохранено как story_cropped.jpg")
else:
    raise Exception(str(response.json()))

Изображение успешно обрезано и сохранено как story_cropped.jpg


In [57]:
from PIL import Image, ImageDraw, ImageFont
#import os
#import openai
#from google.colab import userdata

#os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

def generate_post(topic):
    prompt = f"""
        Используй тему для генерации оригинальной вдохновляющей цитаты.
        Она должна быть краткой, лаконичной и легкой для понимания.
        Цитата должна иметь потенциал изменить чью-то жизнь.
        Максимум 25 слов.
        ---
        Тема: {topic}
        Вдохновляющая цитата:
        """
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].message.content.strip()

# Текст, который нужно добавить
text = generate_post("Кот ниндзя, путь великого воина")
print(text)

"Каждый кот ниндзя знает: истинная сила не в мускулах, а в решимости следовать своему пути, несмотря на преграды."


In [61]:

# Открываем изображение
img = Image.open('story_cropped.jpg').convert('RGBA')

# Создаем объект для рисования на изображении
draw = ImageDraw.Draw(img)

# Задаем шрифт и размер шрифта (обязательно шрифт с поддержкой кириллицы)
#font_path = 'Roboto-Thin.ttf'  # Убедитесь, что шрифт поддерживает кириллицу
font = ImageFont.truetype('FreeMono.ttf', 40)

# Получаем размеры изображения
img_width, img_height = img.size

# Задаем начальную позицию для текста
x, y = 100, 400

# Разбиваем текст на строки по переносу
lines = []
line = ""
for word in text.split():
    if draw.textlength(line + word, font) <= img_width - 200:
        line += word + " "
    else:
        lines.append(line.strip())
        line = word + " "
lines.append(line.strip())

# Получаем высоту строки
ascent, descent = font.getmetrics()
line_height = ascent + descent

# Получаем размеры текста
text_width = max(draw.textlength(line, font) for line in lines)
text_height = line_height * len(lines)

# Создаем новый слой с темно-синим цветом и прозрачностью 70%
overlay = Image.new('RGBA', (int(text_width), int(text_height)), (0, 0, 128, 180))

# Объединяем изображение и слой с помощью наложения
img.paste(overlay, (int(x), int(y)), mask=overlay)

# Добавляем текст на изображение, перенося на новую строку при необходимости
for line in lines:
    draw.text((x, y), line, font=font, fill=(255, 0, 0))
    y += line_height

# Удаляем альфа-канал
img = img.convert("RGB")

# Сохраняем изображение в формате JPEG
img.save("story_text_crop.jpg")